In [ ]:
Model = "gpt-3.5-turbo-1106"
Name = 'mcm_stage_a'
split = 'train'

SKIP_FIRST = 0
Queries = 5
sourcefile = f'./{split}.csv'
save_jsonl = f'./{split}_{Name}.jsonl'
# save_file = f'./{split}_{Name}.csv'

Max_Tokens = 2000
openai_api_key = "your_key"
if '3.5' in Model:
    prices = (0.0015/1000, 0.002/1000) # input, output tokens
    WAIT = 60
elif '4' in Model:
    prices = (0.03/1000, 0.06/1000) # 0.06/0.002= 60/2=30. 
    WAIT = 60


Messages = lambda input: [
    {
      "role": "system",
      "content": "You are an expert lawyer in the domain of U.S. civil procedure. You are given an introduction that describes a legal case, a question about that case and an answer to that question. First, you must identify what legal concept the question is talking about. Note that the given question contains some background on the details of the case. Provide a different background for your question and then ask a multiple choice question in the style of the original question, to illustrate that concept to legal law students. Your output should have the format: Concept:\\n<concept>\\n\\nBackground:\\n<background>\\n\\nQuestion:\\n<question>\\n\\nChoices\\n<choice_A>\\n<choice_B>\\n<choice_C>\\n<choice_D>\\n\\nCorrect Answer:\\n<correct_answer>."
    },
    {
      "role": "user",
      "content": "Introduction:\nThe venue provisions for cases involving corporate defendants are stated in Section 1391(b) of U.S. law. This section clarifies where a case may be brought; either the district where all defendants reside or where a significant portion of the events related to the claim occurred. However, defining where a corporation resides can be complex. Sections 28 U.S.C. §1391(c)(2) and (d) deal with this issue. It defines the residence of a corporation as any judicial district where it is under the court's personal jurisdiction concerning a particular civil action. If a state has more than one judicial district, a corporation is deemed to reside in any district within the state where it has sufficient contacts to be subjected to personal jurisdiction. If there is no such district, the corporation is considered to reside in the district with which it has the most significant contacts. An example is provided, using a fictional corporation and a legal dispute. Multiple potential district venues are considered, shedding light on the application of the provisions.\n\nQuestion:\nChu, a Californian, went skiing at Aspen, in Colorado, which has only one federal judicial district. After he discovered that he was somewhat out of shape, he bought an exercise machine from Jackson, a friend from Kansas with a two-week time share in Aspen, who had brought the machine with him to Aspen after buying it in Texas. The machine was made by Sweda- Trak, a Texas company that sells its products only in Texas, and with its only place of business in the Western District of Texas. Chu brought the machine back to Los Angeles, California, where he was injured using it. Chu brings a diversity action against Jackson and Sweda-Trak in federal court, alleging that Jackson was negligent in maintaining the machine, and Sweda-Trak in designing it. Venue in Chu’s action would be\n\nAnswer:\nproper in the Southern District of California under §1391(b)(2), because a substantial part of the events giving rise to the claim took place there."
    },
    {
      "role": "assistant",
      "content": "Concept:\nVenue in Civil Procedure\n\nBackground:\nSophia, a New York resident, wants to sue Eric, a Texas resident, claiming that he infringed on her copyright by using her work in his recent publication. Sophia has never been to Texas, but she knows that Eric frequently visits New York for business meetings. According to U.S. law, venue refers to the most appropriate location for a trial. It can be the district where the defendant resides or where the events related to the claim occurred.\n\nQuestion:\nIn which district would venue be proper if Sophia brings a copyright infringement lawsuit against Eric in federal court?\n\nChoices:\nA) The Southern District of New York, where Sophia resides\nB) The Western District of Texas, where Eric resides\nC) The Eastern District of New York, where Eric frequently visits for business meetings\nD) The Northern District of Texas, where Sophia's work was originally created\n\nCorrect Answer:\nA) The Southern District of New York, where Sophia resides"
    },
    {
      "role": "user",
      "content": input
    },
  ]


In [ ]:
import os
import openai

openai.api_key = openai_api_key

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(3))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

def query(M):
  '''
  Openai api call
  '''
  response = completion_with_backoff(
      model=Model,
      messages= M,
      temperature=1,
      max_tokens=Max_Tokens,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
  )
  query = response.choices[0].message.content
  cost = prices[0]*response.usage.prompt_tokens + prices[1]*response.usage.completion_tokens
  return query, cost, response # extract_answer(query)

In [ ]:
import re

def get_content(input, start_label, end_label=None):
    start = input.find(start_label)
    if end_label:
        end = input.find(end_label, start)
    else:
        end = len(input)
    content = input[start + len(start_label) : end].strip()
    return ' '.join(content.split())


def get_choices(input):
    pattern = "([A-Z]\))"
    pieces = re.split(pattern, input)[1:]
    choices = [pieces[i+1].strip() for i in range(0, len(pieces), 2)]
    return choices

def answer_collection(input):
    # "Concept:\nVenue in Civil Procedure\n\nBackground:\nSophia, a New York resident, wants to sue Eric, a Texas resident, claiming that he infringed on her copyright by using her work in his recent publication. Sophia has never been to Texas, but she knows that Eric frequently visits New York for business meetings. According to U.S. law, venue refers to the most appropriate location for a trial. It can be the district where the defendant resides or where the events related to the claim occurred.\n\nQuestion:\nIn which district would venue be proper if Sophia brings a copyright infringement lawsuit against Eric in federal court?\n\nChoices:\nA) The Southern District of New York, where Sophia resides\nB) The Western District of Texas, where Eric resides\nC) The Eastern District of New York, where Eric frequently visits for business meetings\nD) The Northern District of Texas, where Sophia's work was originally created\n\nCorrect Answer:\nA) The Southern District of New York, where Sophia resides"
    concept = get_content(input, 'Concept:', 'Background:')
    background = get_content(input, 'Background:', 'Question:')
    question = get_content(input, 'Question:', 'Choices:')
    choices_block = get_content(input, 'Choices:', 'Correct Answer:')
    choices = get_choices(choices_block)
    correct = get_content(input, 'Correct Answer:')

    return concept, background, question, choices, correct

In [ ]:
import pandas as pd
import time
import json

df = pd.read_csv(sourcefile)
cot = []
total=0
total_time=0
# Nones=[]
failed_collection=[]
for index, row in df.iterrows():
    # idx,question,answer,label,analysis,complete analysis,explanation,cot
    intro = row['explanation']
    question = row['question']
    answer = row['answer']
    # label = row['label']
    # explanation = row['analysis']
    
    M = f'Introduction:\n{intro}\n\nQuestion:\n{question}\n\nAnswer:\n{answer}'
    messages = Messages(M)

    tries = 0
    while(tries>=0):
        try:
            ans, cost, response = query(messages)
            break
        except Exception as e:
            wait_min = 5
            print('Met RateLimit exception')
            print(f'Waiting {wait_min} minutes to avoid timeout...')
            time.sleep(wait_min*60)
            total_time+=wait_min*60
            print(f'Total time: {total_time/60:.0f} minutes')
            print('Resuming experiment:\n')
            tries -=1
    total+=1

    
    try:
        concept, background, question, choices, correct = answer_collection(ans)
    except:
        print('Collection Failed!')
        failed_collection.append(row['idx'])
        save_dict = {
            "idx": row['idx'],
            "success": False,
            "prompt": M,
            "output": ans
        }
        with open(save_jsonl, 'a') as f:
            json.dump(save_dict, f)
            f.write('\n')
    else:
        print()
        # save jsonl
        save_dict = {
            "idx": row['idx'],
            "success": True,
            "concept": concept,
            "background": background,
            "question": question,
            "choices": choices,
            "correct_option": correct,
            "intro": intro,
            "prompt": M,
            "output": ans,
        }
        with open(save_jsonl, 'a') as f:
            json.dump(save_dict, f)
            f.write('\n')
            
    finally:
        if total% Queries==0 and WAIT>0:
            if total ==Queries:
                print("Analysis Example:")
                print(M)
                print('-----------------------------------------------------------------------------')
                print(ans)
                print('######')
                print(save_dict)
            print(f'Waiting {WAIT} seconds to avoid timeout...')
            print(f'Total queries: {total}')
            time.sleep(WAIT)
            total_time+=WAIT
            print(f'Total time: {total_time/60:.0f} minutes')
            print('Resuming experiment\n')